<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/main/19_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install hdwallet py_crypto_hd_wallet cryptos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 1.3 MB/s 
     |████████████████████████████████| 96 kB 4.1 MB/s 
     |████████████████████████████████| 69 kB 6.2 MB/s 
     |████████████████████████████████| 62 kB 576 kB/s 
     |████████████████████████████████| 829 kB 50.1 MB/s 
     |████████████████████████████████| 119 kB 62.5 MB/s 
     |████████████████████████████████| 278 kB 44.1 MB/s 
     |████████████████████████████████| 1.3 MB 42.5 MB/s 
     |████████████████████████████████| 871 kB 49.3 MB/s 
     |████████████████████████████████| 2.3 MB 27.0 MB/s 
     |████████████████████████████████| 856 kB 20.5 MB/s 
     |████████████████████████████████| 1.0 MB 65.7 MB/s 
     |████████████████████████████████| 105 kB 58.0 MB/s 
  Created wheel for pysha3: filename=pysha3-1.0.2-cp37-cp37m-linux_x86_64.whl size=133594 sha256=a4ef850b2ab86af35c5e6c8fcb97fc46d079c64c4d1c27763f6f26159d

# Our Key Handing Up Until Now

Real drawbacks:

- To send and recieve coins we would have to create a new private key, public key and address. 
- If we wanted to send coins from one address to another we'd have to create another private key, public key and address.
- If we wanted to send to one another we'd have to send each other our addresses.
- If we wanted to create a multisig wallet together we'd have to share our public keys.

Wouldn't it be nice if we could generate a single private key from which we could generate a whole set of private keys in a deterministic fashion?

Wouldn't it be nice if we could share a single public key (associate with the above single private key) from which we could generate the whole set of associated public keys in a deterministic fashion?

**This better world is possible with hierarchical deterministic wallets (HD wallets)**


Hierarchical deterministic wallets, also know as HD-wallets, are the standard in the cryptocurrency world but applicable for any application where we new to generate and share keys in this fashion.  We will learn how to use them appropriately.

https://pypi.org/project/py-crypto-hd-wallet/



# BIP44 Derivation Paths

| Coin | Symbol | Derivation Path |
| :------ | :----: | ----------:|
|**Bitcoin**| BTC |`m/44'/0'/0'/0/0`|
|**Litecoin**| LTC |`m/44'/2'/0'/0/0`|
|**Dogecoin**| DOGE |`m/44'/3'/0'/0/0`|
|**DigiByte**| DGB |`m/44'/20'/0'/0/0`|
|**Zcash**| ZEC |`m/44'/133'/0'/0/0`|
|**BitcoinCash**| BCH |`m/44'/145'/0'/0/0`|
|**BitcoinSV**| BSV | `m/44'/236'/0'/0/0`|

Bitcoin Improvement Proposal (BIP) 44 defines the standard derivation path for wallets which generate Pay-to-Public-Key-Hash (P2PKH) addresses. BIP 44 also defines the prefixes to be used with associated extended keys.

According to BIP 44, wallets which generate P2PKH addresses should use a derivation path beginning with `m/44'/`. This means the first address generated by a mainnet Bitcoin wallet will have a derivation path of `m/44'/0'/0'/0/0`.



`m/44'/0'/0'/0/0` is labeled as `master / purpose / coin_type / account / change / address_index`

- **master** all HD wallets begin with `m` in the first position
- **purpose** all coins adoptin BIP44  use `44`
- **coin_type** is used to differentiate between different coins 
- **account** is used to have a branch point for different subsets of key pairs
- **change** this field is used to differentiate between receipt addresses and change addresses
- **address_index** this final field is used to iterate through different keypairs/addresses

<div>
<img src="https://github.com/ProfDoeg/Colegio_Invisible/raw/main/img/hd_wallet.png"  width="500"/>
</div>

The brilliant feature of this structure is that there is a hierarchy of public and private keys.

With ONE master private key you can control/generate many private keys for many different coins.  In addition you can share ONE public key to share ALL the derived public keys and addresses.

Say you have a business with a point-of-sale cash register that collects DOGE transactions. Each transaction can have a seperate address. You need only share the public key with the `m/44'/3'/0'/0` derivation path. The POS software can then derive the addresses for `m/44'/3'/0'/0/0`, `m/44'/3'/0'/0/1`, `m/44'/0'/0'/0/2`...etc

For our purposes...all the students can generate a single master private key and share one public key with hq. HQ can then access an array of derived public keys and the associated crypto addresses.

# Mnemonic Seed Phrases

BIP39 is the use of a mnemonic phrase -- a group of easy to remember words -- to serve as a back up to recover your wallet and coins in the event your wallet becomes compromised, lost, or destroyed. This is also known as a mnemonic seed (phrase), recovery phrase, wallet back up, master seed, etc. 

<div>
<img src="https://github.com/ProfDoeg/Colegio_Invisible/raw/main/img/mnemonic.png" width="400"/>
</div>

These words aren't just any words. They are pulled from a specific list of 2048 words known as the BIP39 wordlist. Upon start up, wallets that utilize the BIP39 standard will provide you a 12-24 word phrase randomly chosen from the standard BIP39 wordlist.

Mnemonic seeds are a way of storing the root private key in a human readable format.

The `hdwallet.utils.generate_mnemonic()` function uses a two arguments, strength and language. 

- **strength** 128, 160, 192, 224 or 256
- **language** english, french, italian, spanish, chinese_simplified, chinese_traditional, japanese or korean

In [2]:
from hdwallet.utils import generate_mnemonic
from py_crypto_hd_wallet import HdWalletBipFactory, HdWalletBip44Coins, HdWalletBipChanges
import psutil
import os
import pandas as pd

In [3]:
secret_phrase=generate_mnemonic(language="english", strength=256)
secret_phrase

'top leg more target ketchup access magnet evidence smile warrior jungle scatter border crowd fluid repair coral action casual rookie fresh nest practice load'

In [4]:
hd_wallet_factory = HdWalletBipFactory(HdWalletBip44Coins.DOGECOIN)

In [5]:
my_doge_wallet = hd_wallet_factory.CreateFromMnemonic('DD_wallet', secret_phrase)

In [6]:
my_doge_wallet.Generate(account_idx = 0, change_idx = HdWalletBipChanges.CHAIN_EXT, addr_num = 4)

In [7]:
my_doge_wallet_dict=my_doge_wallet.ToDict()
my_doge_wallet_dict

{'account_idx': 0,
 'account_key': {'address': 'DDPNdR4odtgABZcbb7tuTuufgThVMkdrB3',
  'ex_priv': 'dgpv58Sjpe6yv8BjYUGN8xTHi8wWHrYs5UiZ7XafGG9uUnpokArvrKkU4kTMqjCjn3Htjb1bjZaQBx3j6NAiab6zVg3vbocNnqDN2hv5UkEUgU1',
  'ex_pub': 'dgub8sKmAL2sUGr4qKvXBC5FpzV6kQvVubkGQ8MsRNYtGuspJDydG6JEi3s1QxCkeqh7YxdpozgW4h6dL1DEYU4SW93KF2ZvWWy2cCtWks2u9Ti',
  'raw_compr_pub': '02995458f4ca6714c4b7a16295184869433c14910c8b626ca1d135a64240893b58',
  'raw_priv': '9001d42258ba37de2384861e59a4b2ef1510f848c2662dacc229df1a8bb3da09',
  'raw_uncompr_pub': '04995458f4ca6714c4b7a16295184869433c14910c8b626ca1d135a64240893b588829023c667ff8ad33de1397c330516c621d7070845c4bb66d1ad73bb2da3ef2',
  'wif_priv': 'QTSZLcXWDU4bS2eCGscBt4hbEMJCnEixLKhRdCcJ3gCavcnQQPti'},
 'address': {'address_0': {'address': 'DN5yjNRh1e4Y8yFUS8GmbhVqAhwPqgmCDs',
   'ex_priv': 'dgpv5Ce8AHfCh4m1Zoca8rSoyjMHgKRA8F5cpX8Mjboy3N1m2eFiNrE2gWv9WWDxBZGDocgLfZKLZPuQz4jk8ZDpH8obinuwE2baRXbZ6Z2TtxD',
   'ex_pub': 'dgub8wX9Vyb6FDRLrfGjB64n6att8snnxN7L77uZtiCw

In [8]:
my_doge_wallet_dict.keys()

dict_keys(['wallet_name', 'spec_name', 'coin_name', 'mnemonic', 'passphrase', 'seed_bytes', 'master_key', 'purpose_key', 'coin_key', 'account_idx', 'account_key', 'change_idx', 'change_key', 'address'])

In [9]:
my_doge_wallet_dict['address']['address_1']['wif_priv'] #this is the private key to address 1

'QSUEQKAPqKP2np5BUHzMr3rDFmyhT9gGveykizNXftawsGB1xMWv'

In [10]:
my_doge_wallet_dict['address']['address_1']['raw_uncompr_pub'] #this is the public key to address 1

'04204bc18aedfd5f174ca55ebee3e3380a5df3bc7d256db0058e992fe12ad77aefd8746872c16b2e1a32bcf9c6f285c2964ae876a0de09c2d039fa25e91809b0a6'

In [11]:
my_doge_wallet_dict['change_key']['ex_pub'] #this is the xpub key that can be used to generate all the public keys 

'dgub8tiHZKGcGti4yyNFkvCp4AUFkVxvKBWH2kg6tuSjjCLUtVHopcZKPY7Vfse7E4gsaj8X6jpoNwnMN4R3UFWiyTQzLk9ddaEycRx3LjKsMYf'

In [12]:
def get_addresses(wallet):
    addys=wallet.ToDict()['address']
    return [ addys[addy]['address'] for addy in addys]

def get_privates(wallet):
    addys=wallet.ToDict()['address']
    return [ addys[addy]['wif_priv'] for addy in addys]

def get_publics(wallet):
    addys=wallet.ToDict()['address']
    return [ addys[addy]['raw_uncompr_pub'] for addy in addys]

In [13]:
get_addresses(my_doge_wallet)

['DN5yjNRh1e4Y8yFUS8GmbhVqAhwPqgmCDs',
 'DL4WXCHEKAv26rCj8WcaB9rPjFTK8yEKVh',
 'DCEKUdWy96ppg2X48gHb2vMBKMS6hYh2Ho',
 'DFBkgmycqkE61U8HQ3jKUgyiQ1cujaQi8G']

# File With Some Ex_PubKeys

```text
student,ex_pubkey
00,dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm
01,dgub8tiZR8EbP42Rg4EAn7jYYQRLjCZ2UHg5Bwdb4eASRBvNYwanrTdp8jvN2bHdakaevMj82u9tknJaYvsiaSMpijn7aP2hSsJbLbj8qbuBmZU
02,dgub8tTFHkG1FajRZH2nscUV9166uRMRRAvbeh6zXCctUVgpNwvWpZ16JT1WrSwW46p2hvfPZNCKmb5xQBcUwKBip7iM3tUNMzAkqDDysR1UhbE
03,dgub8tZWJSrqUJ9KDRAQEHU4YH9ToP6f16jLsvi1cP4kA45MRve9PnBM3BPUgPshBoJBS1BebakEw5Qh9W2pAfTWV8YVgSeUfmyjCkScMGabGMK
04,dgub8tP4gsyL1R7U5eZVTDZbA5DF4foatJGq9EikgtaX3rtUp5hZ1QT9CnCxhf1GmceVsH8xjJmzcCMyXdSxekjvQ8M5fZKaNoenMenPDDmPfwM
05,dgub8uqjRZAXa5dvQ2wTaaxTeS45EDd3BbNCo6X1ifCnH2AJvRiH3CsdS3yTqFMvfU3EptHTtDVUb2V7pb5YNaecf5xPkUQ16CWxWpbPLAxo7hL
06,dgub8uTT3NrNwnwMW1pYo4AHR4XSEUf4pZQtHjg1xSuxSBZCvvketHKK3gKcLuiu8BNW9LTgQ5oZs35PT6ToR6xr4Q2PCqfvvoxQ18HSsS2fPcw
```

In [14]:
ex_00='dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm'

In [15]:
watch_00=hd_wallet_factory.CreateFromExtendedKey("watcher_00",ex_00)

In [17]:
watch_00.Generate(account_idx = 0, change_idx = HdWalletBipChanges.CHAIN_EXT, addr_num = 10)

In [18]:
watch_00.IsWatchOnly()

True

In [19]:
watch_00.ToDict()

{'address': {'address_0': {'address': 'DBcNgj9T7zy8b12igfRPQw7XbnaptwqCbm',
   'ex_pub': 'dgub8w6D9esvEam9y5RLQDUvZ8ie96mxQYHeDpq9kBbzKtpT5vcrmCh7BQq1KEJ1aW7m85qQdnynLNxjgd7J3dRG2ZAfWAupuGDKrx5EQTsQKVH',
   'raw_compr_pub': '03797a01b3e9c7ac5273ab0ff37c0917dfe0f7909ec2144977aad5f2d96bae1fc7',
   'raw_uncompr_pub': '04797a01b3e9c7ac5273ab0ff37c0917dfe0f7909ec2144977aad5f2d96bae1fc74f41335fc8ae94be4d55daaf3afed7af3057dcb44aede7ca9d9888a02ed62da9'},
  'address_1': {'address': 'DTAvSJgxbQ7hQSffTB5tg9MPZzJ6VKLQGJ',
   'ex_pub': 'dgub8w6D9esvEamA1HhaMSogPDau7oWhnThpS2phDqstSoJJKdUD5iZRQXPZSeP2mM2gkVxeSadkUSeqhN98HDYBEziAU2oad46F4osv58MnFwZ',
   'raw_compr_pub': '03bf03dea1cb1dddb81f968d3828e6da8f6e34447c507c6add66b54f59551a2afb',
   'raw_uncompr_pub': '04bf03dea1cb1dddb81f968d3828e6da8f6e34447c507c6add66b54f59551a2afbb3c676fb42d6e71ea3073a006d6073bb023de3d987fa4cb4373281fb9e8aac8f'},
  'address_2': {'address': 'DJvdop5eker2ZHij9Gmh1qGCc7G2TijPy3',
   'ex_pub': 'dgub8w6D9esvEamA5TFkyAF4Yh5ixf

In [20]:
get_addresses(watch_00)

['DBcNgj9T7zy8b12igfRPQw7XbnaptwqCbm',
 'DTAvSJgxbQ7hQSffTB5tg9MPZzJ6VKLQGJ',
 'DJvdop5eker2ZHij9Gmh1qGCc7G2TijPy3',
 'DRsTTHnf9SYiMt8vinTfNDi78Ag55te62e',
 'DPjudXU9XHz487RpXcAfSPbY4h1F85cyzd',
 'DJuRujqNU6yYRx7TUPhdSBomnyjq3fJvUF',
 'D8XbpiDauFQftCbf1gy8FTUHKQKm9KcfuX',
 'D8VhDPNQzsHCqrLkPWERVJyxT4SoLZNSzL',
 'DFb5pALcHZrTtFdRVKma8rWM73XZADQWRN',
 'DDpZQ8uPNXMrcADuY2fphnHgYeVwpD5CA2']